In [32]:
import imageio
import os
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import torchvision
import shutil
from tqdm import tqdm
import torch.nn as nn

import sys

import torch.optim as optim


import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import sys
import os
from scipy import stats

import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader


sys.path.insert(0, '/home/riccardo/Desktop/OcclusionInference/Architectures')

from models.BLT_models import multi_VAE
from data_loaders.dataset_sup import MyDataset_encoder, MyDataset_decoder
from data_loaders.dataset_unsup import MyDataset_unsup

from models.Lin_model import Lin_model 

from solvers.utils_mod import DataGather, get_accuracy
from solvers.losses import supervised_encoder_loss


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
net_1 = multi_VAE('BLT','BLT', 12, 0 ,32, 1, 4, False, 4, 1, False)
file_path = '/home/riccardo/Desktop/Experiments/AE/Unfrozen/2_digts/BLT_BLT_depth_zdim12_2/main/last'
checkpoint = torch.load(file_path) 
net_1.load_state_dict(checkpoint['model_states']['net'])


using BLT encoder BLT decoder


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [26]:


def traverse_z(NN, example_id, ID, output_dir, global_iter, sbd ,num_frames = 100 ):
    z_dim = NN.z_dim_tot
    
    x_test_sample = example_id['x']
    y_test_sample = example_id['y']
    x_test_sample = torch.unsqueeze(x_test_sample, 0)
    
    #encode a sample image
    z_distributions = NN._encode(x_test_sample)
    z_distributions = z_distributions[-1]
    z_sample = z_distributions[:, :NN.z_dim_bern]
   
    num_slice = int(1000/num_frames)


    dist_samples = np.random.uniform(low=-6, high=6, size=1000)
    dist_samples.sort()
    dist_samples = torch.from_numpy(dist_samples[0::num_slice])

            
    traverse_input = torch.mul(torch.ones(num_frames*z_dim,1),z_sample)
    
     
    #print(traverse_input.shape)

    indexs = np.arange(0, num_frames*z_dim, num_frames)
    for i in indexs:
        z = int(i/num_frames)
        traverse_input[i:(i+num_frames),z] = dist_samples
   
    #create all reconstruction images
    x_recon = NN._decode(z_sample)
    x_recon = x_recon[-1]
    reconst = NN._decode(traverse_input)
    reconst = reconst[-1]
    print(reconst.shape)

    #Create GIFs
    indexs = np.arange(0, num_frames*z_dim, num_frames)
    for i in indexs:
        #save images for each gif into the images list
        images = []
        for e in range(num_frames):
            #save images to make gifs into different folders
            filename = '{}/traversals{}_{}/z{}/img{}.png'.format(output_dir,global_iter,ID,int(i/num_frames),e)
            directory = os.path.dirname(filename)
            if not os.path.exists(directory):
                os.makedirs(directory)
            torchvision.utils.save_image(F.sigmoid(reconst[i+e,0,:,:].cpu()) , filename)
            images.append(imageio.imread(filename))


        #save all gifs into same folder
        filename_2 = '{}/traversals_gifs{}_{}/traversing_z_{}.gif'.format(
            output_dir,global_iter, ID,int(i/num_frames),int(i/num_frames))
        directory_2 = os.path.dirname(filename_2)
        if not os.path.exists(directory_2):
                os.makedirs(directory_2)
        imageio.mimsave('{}/traversals_gifs{}_{}/traversing_z_{}.gif'.format(
            output_dir, global_iter, ID, int(i/num_frames),int(i/num_frames)), images)
        
        with open('{}/traversals_gifs{}_{}/encoded_z.txt'.format(output_dir,global_iter,ID), 'w') as f:
            f.write(str(z_sample.detach().numpy()))
        
        #add the reconstruction image to the GIF image folder
        torchvision.utils.save_image(F.sigmoid(x_recon[0,0,:,:]),
                                        '{}/traversals_gifs{}_{}/recon.png'.format(output_dir,global_iter,ID))
        #add the actual target image to the GIF image folder
        torchvision.utils.save_image(y_test_sample[0,:,:],
                                        '{}/traversals_gifs{}_{}/target.png'.format(output_dir,global_iter,ID))
        shutil.rmtree(directory)
            

In [34]:
train_image_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/train/orig/"
train_target_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/digts.csv"

gnrl_image_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/gnrl/orig/"
gnrl_target_paths = "/home/riccardo/Desktop/Data/100k_2digt_BWE_2/digts/gnrl/inverse/"

dset = MyDataset_unsup
train_data_size = len(os.listdir(train_image_paths))

encoder_target_type = 'depth_ordered_one_hot_xy'

gnrl_data = dset(gnrl_image_paths,gnrl_target_paths, 32, 'gnrl',train_data_size )

Sorting gnrl image files


In [35]:
output_dir = '/home/riccardo/Desktop/Experiments/traverse_AE/border2'

for i in range(3):
    example_id = gnrl_data.__getitem__(i)
    traverse_z(net_1,example_id,  ID=str(i),output_dir=output_dir,global_iter=100000,  sbd = False, num_frames=100  )


torch.Size([1200, 1, 32, 32])


/home/riccardo/miniconda3/envs/rconci/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([1200, 1, 32, 32])
torch.Size([1200, 1, 32, 32])
